In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../..')
from JKBio import TerraFunction as terra
%load_ext autoreload
from JKBio import Helper as h
%autoreload 2
import pickle
%load_ext rpy2.ipython
#%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
import itertools

from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import gseapy
from JKBio.helper import pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
! gsutil -m mv gs://transfer-amlproject/*.fastq.gz gs://transfer-amlproject/RNPv3/

In [ ]:
sampleset='RNPv3'

In [ ]:
terra.uploadFromFolder('transfer-amlproject','RNPv3/',
                       'broad-firecloud-ccle/hg38_RNAseq',samplesetname=sampleset,
                      fformat="fastqR1R2",loc=1)

In [ ]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/hg38_RNAseq')

In [ ]:
submission_id = wm.create_submission("star_v1-0_BETA_cfg", sampleset, 'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
submission_id = wm.create_submission("rsem_v1-0_BETA_cfg", 
                                      sampleset,'sample_set',expression='this.samples')
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
submission_id = wm.create_submission("rsem_aggregate_results_v1-0_BETA_cfg", 
                                         samplesetname)
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

In [ ]:
results = wm.get_sample_sets().loc[sampleset]
rsem_genes_expected_count = results['rsem_genes_expected_count']

In [ ]:
results

In [ ]:
! gsutil cp $rsem_genes_expected_count data/

In [ ]:
file = 'data/'+rsem_genes_expected_count.split('/')[-1]

In [ ]:
file

In [ ]:
! gunzip $file

In [ ]:
rsem_genes_expected_count = pd.read_csv(file[:-3], sep='\t')

In [ ]:
data = rsem_genes_expected_count.drop("transcript_id(s)",1)

In [ ]:
data["gene_id"] = convertGenes(data['gene_id'])[0]

In [ ]:
data=data.set_index('gene_id')

In [ ]:
experiments = set([i.split('-')[0] for i in data.columns])
experiments

## pre processing 

filter some more

In [ ]:
toremove = np.argwhere(data.values.var(1)==0)
toremove.ravel()

In [ ]:
toremove.shape

In [ ]:
data = data.drop(data.iloc[toremove.ravel()].index,0)

In [ ]:
data.shape

renormalize the data

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
genenames = data.index
ctfpos = [i for i, val in enumerate(genenames) if val in ctf]
notctfpos = [i for i, val in enumerate(genenames) if val not in ctf]

We find a CTF not in the dataset

In [ ]:
[val for val in ctf if val not in genenames]

In [ ]:
ctf.remove('IKAROS')

## DESEQ ANALYSIS

In [ ]:
expe = {}
for val in experiments: 
    expe[val] = [1 if val in col else 0 for col in data.columns]

In [ ]:
data = data.reset_index()

In [ ]:
experiments.remove('Non')
results = {}

In [ ]:
experiments - set(ctf)

In [ ]:
experiments

In [ ]:
for val in experiments:  
    design = pd.DataFrame(index=data.columns[1:], columns=['DMSO','Target'], 
                          data=np.array([expe['Non'],expe[val]]).T)
    design.index = design.index.astype(str).str.replace('-','.')
    deseq = pyDESeq2.pyDESeq2(count_matrix=data, design_matrix = design, 
                              design_formula='~DMSO + Target', gene_column="gene_id")
    deseq.run_deseq()
    deseq.get_deseq_result()
    r = deseq.deseq_result
    r.pvalue = np.nan_to_num(np.array(r.pvalue), 1)
    r.log2FoldChange = np.nan_to_num(np.array(r.log2FoldChange), 0)
    results[val] = r

## what are most downregulated (volcano)

In [ ]:
MS2res.pvalue = np.nan_to_num(np.array(MS2res.pvalue), 1)
MS2res.log2FoldChange = np.nan_to_num(np.array(MS2res.log2FoldChange), 0)
MS2res.gene_id = convertGenes(MS2res.gene_id)[0]
show(volcano(MS2res,tohighlight=ctf))

## CTF (volcano)

In [ ]:
results

In [ ]:
for val in experiments:
    a = volcano(results[val],tohighlight=ctf,title=val)
    try:
        show(a)
    except RuntimeError:
        show(a)

In [ ]:
data = data.set_index('gene_id')

## any bias in the data

In [ ]:
red = PCA(50).fit_transform(data.T)
a = scatter(red, labels=data.columns)

In [ ]:
red.shape

In [ ]:
red = TSNE(2,10).fit_transform(red)

In [ ]:
a = scatter(red[:,:2], labels=data.columns, radi=10)

In [ ]:
show(a)

In [ ]:
dataclean = data[data.max(1) > 10].drop(['LMO2-r1','LMO2-r2','LMO2-r3'],1)

In [ ]:
for val in grouped(dataclean.columns,3):
    val = list(val)
    dataclean[val[0].split('-')[0]] = dataclean[val].mean(1)
    dataclean = dataclean.drop(val,1)

In [ ]:
pca = PCA(20)
red = pca.fit_transform(dataclean.T)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
a = scatter(red[:,:2], labels=dataclean.columns, radi=4000, title='batch effect RNP')

MAX CEBPa ZEB2 

In [ ]:
show(a)

In [ ]:
save(a,'data/plots/batcheffect_RNP.html')

## GSEA analysis

In [ ]:
res = {}

In [ ]:
for val in experiments:
    print(val)
    totest = data[[v for v in data.columns if val in v or 'Non' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='WikiPathways_2013', 
                cls= cls, no_plot=False, processes=6)
    res[val].res2d['Term'] = [i.split('_')[0][:50] for i in res[val].res2d.index]
    
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
with open('wikipathway', 'wb') as f:
    pickle.dump(res,f)

In [ ]:
with open('wikipathway','rb') as f:
    res = pickle.load(f)

In [ ]:
for i, val in enumerate(experiments):
    plt.figure(i)
    res[val].res2d.Term = [' '.join(i.split('_')[1:-2]) for i in res[val].res2d.index]
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.index))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[i][n] = v.es
res = pd.DataFrame(a, index=res.keys())
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
model = AgglomerativeClustering(n_clusters=6,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(res)
ii = itertools.count(res.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = plotCorrelationMatrix(res.values[sort],res.index[sort].tolist(),interactive=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
fig.savefig("enriched_terms.png")

In [ ]:
show(a)

In [ ]:
fi

In [ ]:
experiments

In [ ]:
data

In [ ]:
for i, val in enumerate(experiments):
    print(val)
    totest = data[[v for v in data.columns if val in v or 'Non' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='GO_Biological_Process_2015', 
                cls= cls, no_plot=False, processes=6)
    res[val].res2d['Term'] = [i.split('_')[0][:50] for i in res[val].res2d.index]
    plt.figure(i)
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
with open('GO_Biological_Process_2015', 'wb') as f:
    pickle.dump(res,f)

In [ ]:
with open('GO_Biological_Process_2015','rb') as f:
    res = pickle.load(f)

creating matrices

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.Term))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[v.Term][n] = v.es
res = pd.DataFrame(a, index=res.keys())
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
model = AgglomerativeClustering(n_clusters=5,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(res)
ii = itertools.count(res.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]

In [ ]:
sort = labels.argsort()

In [ ]:
a = plotCorrelationMatrix(res.values[sort],res.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

In [ ]:
cluster1= ['LMO2','LYL1','MAX','MEF2C']
cluster2=['GFI1','FLI1','MYB','IKZF1','ELF2','CEBPa','MEIS1']
cluster3=['IRF2BP2','MEF2C','CDK6','MEF2D','IRF8','BRD4','MYC']
cluster4= ['RUNX1','RUNX2','ZMYND8']

In [ ]:
res.loc[cluster2].mean().sort_values()

In [ ]:
'GO_Molecular_Function_2015',
'GeneSigDB',
'ENCODE_TF_ChIP-seq_2014',
#'Drug_Perturbations_from_GEO_2014',
'GO_Cellular_Component_2015',
'GO_Biological_Process_2015',
'PPI_Hub_Proteins',
'WikiPathways_2013',
'TF-LOF_Expression_from_GEO',
# msig db C2 C6 H http://software.broadinstitute.org/gsea/msigdb/annotate.jsp
# max's crc

## compare to the drop of CTF

In [ ]:
ctf = [
 'BRD4',
 'CDK6',
 'CEBPA',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'
]

In [ ]:
deseq = pd.DataFrame()
for k, val in results.items():
    deseq[k] = val.log2FoldChange
deseq=deseq.T

In [ ]:
deseq

In [ ]:
a = plotCorrelationMatrix(a,
                          deseq.index[sort].tolist(),interactive=True)

In [ ]:
ctf[11] = 'CEBPa'
ctf[]

In [ ]:
ctf

dropping ETV6 SP1  GSE1 LDB1

In [ ]:
deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'RUNX2',
 'ZMYND8']]

In [ ]:
show(a)

In [ ]:
deseq_ctf = deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'ZMYND8']]
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq_ctf)
ii = itertools.count(deseq_ctf.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq_ctf.values[sort]
a = plotCorrelationMatrix(a, deseq_ctf.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

In [ ]:
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq)
ii = itertools.count(deseq.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq.values[sort]
a = plotCorrelationMatrix(a, deseq.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

tsne, pca, clustering accross TF, CRC, most var genes, both ways.